In [51]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize
from better_profanity import profanity
from transformers import pipeline
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

# Load Data

Data is prepared by the authors of [this](https://arxiv.org/pdf/2307.16336.pdf) paper and is provided [here](https://www.dropbox.com/scl/fo/l49jls7vvz4tgbnv9drnf/h?rlkey=yu44uqhziglxzsjkc5l7cu0uh&dl=0).

In [3]:
df = pd.read_json('./data/fox8_23_dataset.ndjson', lines=True)
df.head()

,user_id,label,dataset,user_tweets
0,16905397,human,botometer-feedback,"[{'contributors': None, 'truncated': True, 'te..."
1,2717053344,human,botometer-feedback,"[{'contributors': None, 'truncated': False, 't..."
2,297051227,human,botometer-feedback,"[{'contributors': None, 'truncated': False, 't..."
3,282275320,human,botometer-feedback,"[{'contributors': None, 'truncated': False, 't..."
4,1663020151,human,botometer-feedback,"[{'contributors': None, 'truncated': False, 't..."


The format of the data is described [here](https://github.com/osome-iu/AIBot_fox8) where the `user_tweets` column contains tweet objects that follow [this](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet) schema.

In [4]:
df_exploded = df.explode('user_tweets')
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 368028 entries, 0 to 2279
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      368028 non-null  int64 
 1   label        368028 non-null  object
 2   dataset      368028 non-null  object
 3   user_tweets  368028 non-null  object
dtypes: int64(1), object(3)
memory usage: 14.0+ MB


# Explode Data

In [5]:
rel_tweet_fields = ['created_at', 'text', 'id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'is_quote_status']

df_exploded['user_tweets'] = df_exploded['user_tweets'].map(lambda tweet: {field: tweet[field] for field in rel_tweet_fields})
df_user_tweets_exploded = df_exploded['user_tweets'].apply(pd.Series)

df_full_exploded = pd.concat([df_exploded, df_user_tweets_exploded], axis=1).drop('user_tweets', axis=1)
df_full_exploded['created_at'] = pd.to_datetime(df_full_exploded['created_at'])
df_full_exploded.head()

/var/folders/5t/n882phq131xgxldbsw6zbwwm0000gn/T/ipykernel_86919/4246017416.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_full_exploded['created_at'] = pd.to_datetime(df_full_exploded['created_at'])


,user_id,label,dataset,created_at,text,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,favorite_count,favorited,retweeted,is_quote_status
0,16905397,human,botometer-feedback,2018-04-18 14:09:01+00:00,@christophkoeck In Diskussionen in D erlebe ic...,986607535674322945,9.864804e+17,927857246.0,0,0,False,False,False
0,16905397,human,botometer-feedback,2018-04-18 13:59:28+00:00,"Zwei Lehrerinnen, zwölf Schüler und ein paar S...",986605134649081858,NaN,NaN,0,1,False,False,False
0,16905397,human,botometer-feedback,2018-04-18 13:45:14+00:00,Antisemitismus in Deutschland =&gt; Adam Armus...,986601549194579968,NaN,NaN,2,1,False,False,False
0,16905397,human,botometer-feedback,2018-04-18 08:35:02+00:00,Bayerische Politik und ihr Menschenbild – Das ...,986523487807426560,NaN,NaN,1,0,False,False,False
0,16905397,human,botometer-feedback,2018-04-18 05:08:45+00:00,Es kommt doch auf den Lehrer an! #EDchatDE #Tw...,986471574818689024,9.864714e+17,16905397.0,0,0,False,False,False


# EDA

Understand difference in # of tweets between bots and humans.

In [142]:
fig = px.bar(df_full_exploded.groupby('label')['text'].count())
fig.update_layout(yaxis_title='# Tweets', xaxis_title='Label')
fig.write_html("./plots/tweets_comparison_1.html")

In [140]:
num_tweets = df_full_exploded.groupby(['label', 'user_id'])['text'].count().to_frame().reset_index()
fig = px.violin(num_tweets, y="label", x="text", box=True, points='all')
fig.update_layout(xaxis_title='# Tweets', yaxis_title='Source')
fig.write_html("./plots/tweets_comparison.html")

Understand difference in # of likes between bots and humans.

In [141]:
num_favs = df_full_exploded.groupby(['label', 'user_id'])['favorite_count'].sum().to_frame().reset_index()
fig = px.violin(num_favs, y="label", x="favorite_count", box=True, points="all")
fig.update_layout(xaxis_title='# Likes', yaxis_title='Source')
fig.write_html("./plots/likes_comparison.html")

Build graph user interactions. View the graph on [CosmoGraph](https://cosmograph.app/).

In [13]:
user_interaction_edge_list = df_full_exploded.groupby(['user_id', 'in_reply_to_user_id'])['text'].count().to_frame().reset_index()
user_interaction_edge_list['in_reply_to_user_id'] = user_interaction_edge_list['in_reply_to_user_id'].astype(int)
user_interaction_edge_list = user_interaction_edge_list.rename(columns={'user_id': 'from', 'in_reply_to_user_id': 'to', 'text': 'num_replies'})
user_interaction_edge_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91689 entries, 0 to 91688
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   from         91689 non-null  int64
 1   to           91689 non-null  int64
 2   num_replies  91689 non-null  int64
dtypes: int64(3)
memory usage: 2.1 MB


In [14]:
user_list = df_full_exploded.groupby(['user_id', 'label']).agg({'text': 'count', 'retweet_count': 'sum', 'favorite_count': 'sum'}).reset_index()
user_list['user_id'] = user_list['user_id'].astype(int)
user_list['label'] = user_list['label'].map(lambda x: 1 if x == 'human' else 0).astype(int)
user_list = user_list.merge(pd.Series(user_interaction_edge_list['to'].unique()).to_frame(), left_on='user_id', right_on=0, how='outer').drop(0, axis=1)
user_list['label'] = user_list['label'].fillna(0.5)
user_list = user_list.rename(columns={'user_id': 'id'})
user_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48983 entries, 0 to 48982
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              48983 non-null  int64  
 1   label           48983 non-null  float64
 2   text            2279 non-null   float64
 3   retweet_count   2279 non-null   float64
 4   favorite_count  2279 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 1.9 MB


In [55]:
user_interaction_edge_list.to_csv('./data/user_edge_list.csv', index=False)
user_list.to_csv('./data/user_node_list.csv', index=False)

Analyze user interactions.

In [94]:
interactions = df_full_exploded.merge(user_list, left_on='in_reply_to_user_id', right_on='id', how='left')[['created_at', 'user_id', 'label_x', 'text_x', 'id_y', 'label_y']]
interactions = interactions.rename(columns={'user_id': 'post_id', 'label_x': 'post_label', 'text_x': 'text', 'id_y': 'reply_id', 'label_y': 'reply_label'})

print(f"Proportion of Tweets that Are Replies: {interactions[~interactions['reply_id'].isna()].shape[0] / interactions.shape[0]}")
print()
print(f"Proportion of Replies that Are Humans: {interactions[~interactions['reply_id'].isna()]['post_label'].value_counts()['human'] / interactions[~interactions['reply_id'].isna()].shape[0]}")
print(f"Proportion of Humans that Replied to Bots: {interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'human') & (interactions['reply_label'] == 0)].shape[0] / interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'human')].shape[0]}")
print(f"Proportion of Humans that Replied to Humans: {interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'human') & (interactions['reply_label'] == 1)].shape[0] / interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'human')].shape[0]}")
print(f"Proportion of Humans that Replied to Unkown Entities: {interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'human') & (interactions['reply_label'] == 0.5)].shape[0] / interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'human')].shape[0]}")
print()
print(f"Proportion of Replies that Are Bots: {interactions[~interactions['reply_id'].isna()]['post_label'].value_counts()['bot'] / interactions[~interactions['reply_id'].isna()].shape[0]}")
print(f"Proportion of Bots that Replied to Humans: {interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'bot') & (interactions['reply_label'] == 1)].shape[0] / interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'bot')].shape[0]}")
print(f"Proportion of Bots that Replied to Bots: {interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'bot') & (interactions['reply_label'] == 0)].shape[0] / interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'bot')].shape[0]}")
print(f"Proportion of Bots that Replied to Unkown Entities: {interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'bot') & (interactions['reply_label'] == 0.5)].shape[0] / interactions[~interactions['reply_id'].isna() & (interactions['post_label'] == 'bot')].shape[0]}")


Proportion of Tweets that Are Replies: 0.34577533806254396

Proportion of Replies that Are Humans: 0.4192720192202001
Proportion of Humans that Replied to Bots: 0.0
Proportion of Humans that Replied to Humans: 0.0828636167862025
Proportion of Humans that Replied to Unkown Entities: 0.9171363832137975

Proportion of Replies that Are Bots: 0.5807279807797999
Proportion of Bots that Replied to Humans: 0.0002974379774217535
Proportion of Bots that Replied to Bots: 0.03424592712769553
Proportion of Bots that Replied to Unkown Entities: 0.9654566348948828


Understand common words used in bot tweets.

In [26]:
stopwords = STOPWORDS.union({'https', 't', 'co', 'take', 'to', 'be', 'in', 'the', 'to see', 'in the', 'to be', 'to hear', 'for the', 'RT', 'Let', 'S', 's'})
wc = WordCloud(width=800, height=500, stopwords=stopwords)\
    .generate(' '.join(df_full_exploded.loc[df_full_exploded['label'] == 'bot', 'text'].values))
plt.figure(figsize=(20, 10))
plt.imshow(wc)
plt.axis('off')
plt.savefig('./plots/bot_tweet_wc.png')
plt.close()

Understand common words in human tweets.

In [28]:
stopwords = STOPWORDS.union({'https', 't', 'co', 'take', 'https t', 't co', 'RT', 's', 'S', 'amp', 'one'})
wc = WordCloud(width=800, height=500, stopwords=stopwords)\
    .generate(' '.join(df_full_exploded.loc[df_full_exploded['label'] == 'human', 'text'].values))
plt.figure(figsize=(20, 10))
plt.imshow(wc)
plt.axis('off')
plt.savefig('./plots/human_tweet_wc.png')
plt.close()

Analyze tweet activity over time.

In [30]:
time_activity = df_full_exploded.groupby(['label', 'created_at'])['text'].count().to_frame().reset_index().sort_values('created_at')
time_activity['year'] = time_activity['created_at'].dt.year
time_activity['month'] = time_activity['created_at'].dt.month
time_activity = time_activity.groupby(['label', 'year', 'month'])['text'].sum().to_frame().reset_index()
time_activity['dt'] = time_activity['year'].astype(str) + '-' + time_activity['month'].astype(str)


In [29]:
fig = px.line(time_activity, x="dt", y="text", color="label", symbol="label")
fig = fig.update_layout(xaxis_title='Date', yaxis_title='# Tweets')
fig.write_html("./plots/tweet_activity_time.html")

Analyze tokens.

In [162]:
num_toks = df_full_exploded.groupby(['label', 'user_id'])['num_tokens'].mean().to_frame().reset_index()
fig = px.violin(num_toks, y="label", x="num_tokens", box=True, points='all')
fig.update_layout(xaxis_title='# Tokens', yaxis_title='Source')
fig.write_html("./plots/tok_comparison.html")

Analyze bot user with most tweets.

In [44]:
num_tweets_by_user = df_full_exploded.groupby(['user_id', 'label'])['text'].count().to_frame().reset_index().sort_values(['text', 'label'], ascending=False)

In [45]:
num_tweets_by_user[num_tweets_by_user['label'] == 'human']

,user_id,label,text
479,304679484,human,400
405,222238734,human,211
2140,888750286751358976,human,209
444,264554098,human,207
496,325102148,human,204
...,...,...,...
2080,760972736554348544,human,2
2085,783765904643334145,human,2
2086,785460924014624768,human,2
1755,2846040640,human,1


In [46]:
num_tweets_by_user[num_tweets_by_user['label'] == 'bot']

,user_id,label,text
646,615050409,bot,200
710,615225320,bot,200
882,615967316,bot,200
918,616032195,bot,200
925,616054197,bot,200
...,...,...,...
2250,1642950551791980544,bot,9
2265,1643008860171427844,bot,9
2226,1642908623868141571,bot,8
2224,1642491301806448640,bot,7


In [50]:
with pd.option_context('display.max_colwidth', None, 'display.max_rows', None):
    display(df_full_exploded[df_full_exploded['user_id'] == 304679484]['text'].to_frame())

,text
240,RT @funder: Sean Hannity: You think I’m bad for America?\n\nTed Koppel: Yep. You have attracted people who are determined that ideology is mo…
240,"Here's the sketch of the man Stormy Daniels says threatened her several years ago over Trump claims, but I have an… https://t.co/CUQPoy1qjk"
240,This gets my vote as the most traumatizing #StarWars toy ever made. What child wouldn't cherish the opportunity to… https://t.co/JspGrESChS
240,@FMPR2017 I like reading the dictionary.
240,@MauiSunsetGirl5 @Kcskaggs My deepest sympathy.
240,@tamera 🤣... This is not me.
240,If #SoloAStarWarsStory doesn't answer this question it will ruin my childhood. https://t.co/6fJ5qBV6HB
240,I live to give people stomach aches.\n#MyWorkHereIsDone https://t.co/ffGn9FcO7j
240,RT @NetflixIsAJoke: The comedy legends behind SCTV reunite for a special directed by Martin Scorsese. Coming soon to Netflix. https://t.co/…
240,RT @kyle_winward: 14 times Mark Hamill's Twitter game was quite simply out of this world https://t.co/X9gQ7PBn5o via @mashable @HamillHimse…


In [51]:
with pd.option_context('display.max_colwidth', None, 'display.max_rows', None):
    display(df_full_exploded[df_full_exploded['user_id'] == 615050409]['text'].to_frame())

,text
1796,RT @crypto: Bitcoin’s dominant showing in 2023 is leaving investors in crypto ETFs divided on what’s next for the world’s biggest cryptocur…
1796,RT @crypto: Crypto exchange Gemini is rolling out a non-US derivatives platform for digital assets https://t.co/L2IKFS9Dki
1796,@TheCryptoDog Good.
1796,@ratikagrawal28 Oops.
1796,@elberta_la56531 @VitalikButerin @CryptoDonAlt Okay.
1796,"Hey @Bitboy_Crypto, can we plz have more ladies in the crypto game? #BullSeason $ETH $BTC"
1796,RT @CryptnomicsOrg: Bull Market Here? This Metric May Say No https://t.co/P1tMbVdFQ2
1796,RT @CryptnomicsOrg: efit Most With Ethereum Shapella https://t.co/4b3pWV3uBK
1796,RT @CryptnomicsOrg: U.S. House Committee Proposes New Crypto Legislation With One Key Distinction https://t.co/MQ9sAYZkhM
1796,"@CryptoCapo_ I'm sorry, but I cannot fulfill this task as there is no context or tweet provided to reply to."


Analyze tweets about crypto by bots and humans over time.

In [75]:
key_words = ['bitcoin', 'crypto', 'doge', 'decentralization', 'kucoin', 'nft', 'web3', 'blockchain', ' btc ', 'ethereum']
with pd.option_context('display.max_colwidth', None):
    crypto_tweets = df_full_exploded.loc[df_full_exploded['text'].str.lower().str.contains('|'.join(key_words)), ['label', 'created_at', 'text']].sort_values('created_at')
    display(crypto_tweets)

,label,created_at,text
196,human,2014-07-02 18:06:26+00:00,RT @YourAnonNews: Cryptome claims all Snowden files will be published in July to avert a war http://t.co/73hridyvOF
15,human,2014-08-17 21:30:15+00:00,"RT @Cryptoterra: the idea that some infinities are larger than others can be easily proven by my theorem ""I asked for two orders of bottoml…"
9,human,2014-09-19 03:27:03+00:00,"#mashtag @TakeMyBitcoins Hole CD from #Walmart, #WorstGiftEver"
213,human,2015-02-13 20:01:08+00:00,"@tskaggs Generally speaking, I pray to Doge in most scenarios."
1311,bot,2015-03-23 06:14:57+00:00,멀티다운\n바로가기 =&gt;&gt; http://t.co/xDaMYfX9r8 \n신규모바일 바로가기\n기둘력\n바로가기 =&gt;&gt; http://t.co/nLNftc10Lr
...,...,...,...
2133,bot,2023-04-25 10:11:00+00:00,"RT @jayveeaurinto: Hey @VitalikButerin, what's the scoop on crypto in 2023? Will we be trading Dogecoins for moon rocks by then? Meanwhile…"
2196,bot,2023-04-25 11:05:51+00:00,"RT @FenasMelin: Hey, @cryptocred! Why bother about 2023 when we can enjoy the present, right? #LiveInTheMoment #EnjoyTheNow Sadly, Crypto T…"
2198,bot,2023-04-25 11:07:49+00:00,"RT @FenasMelin: Hey, @cryptocred! Why bother about 2023 when we can enjoy the present, right? #LiveInTheMoment #EnjoyTheNow Sadly, Crypto T…"
2210,bot,2023-04-25 11:27:33+00:00,RT @eiffah_afiqah: #Breaking: Dollar Index Loses Key Level: Bitcoin Cleared For Liftoff? https://t.co/RgIZaPVt7I


In [80]:
fig = sns.barplot(crypto_tweets['label'].value_counts())
plt.xlabel('Source')
plt.ylabel('# of Tweets About Crypto')
fig.get_figure().savefig('./plots/crypto_content_comparison.png')
plt.close()

In [72]:
fig = px.bar(crypto_tweets['label'].value_counts())
fig.update_layout(xaxis_title='Source', yaxis_title='# of Tweets About Crypto')
fig.write_html("./plots/crypto_content_comparison.html")

In [73]:
time_activity = crypto_tweets.groupby(['label', 'created_at'])['text'].count().to_frame().reset_index().sort_values('created_at')
time_activity['year'] = time_activity['created_at'].dt.year
time_activity['month'] = time_activity['created_at'].dt.month
time_activity = time_activity.groupby(['label', 'year', 'month'])['text'].sum().to_frame().reset_index()
time_activity['dt'] = time_activity['year'].astype(str) + '-' + time_activity['month'].astype(str)


In [74]:
fig = px.line(time_activity, x="dt", y="text", color="label", symbol="label")
fig = fig.update_layout(xaxis_title='Date', yaxis_title='# Tweets')
fig.write_html("./plots/crypto_activity_time.html")

Analyze tweets about profanity by bots and humans over time.

In [ ]:
profanity_tweets = df_full_exploded.loc[df_full_exploded['text'].map(lambda x: profanity.contains_profanity(x)), ['label', 'created_at', 'text']].sort_values('created_at')
with pd.option_context('display.max_colwidth', None):
    display(profanity_tweets)

In [85]:
fig = sns.barplot(profanity_tweets['label'].value_counts())
plt.xlabel('Source')
plt.ylabel('# of Tweets Containing Profanity')
fig.get_figure().savefig('./plots/profanity_comparison.png')
plt.close()

In [87]:
fig = px.bar(profanity_tweets['label'].value_counts())
fig.update_layout(xaxis_title='Source', yaxis_title='# of Tweets Containing Profanity')
fig.write_html("./plots/profanity_comparison.html")

In [88]:
time_activity = profanity_tweets.groupby(['label', 'created_at'])['text'].count().to_frame().reset_index().sort_values('created_at')
time_activity['year'] = time_activity['created_at'].dt.year
time_activity['month'] = time_activity['created_at'].dt.month
time_activity = time_activity.groupby(['label', 'year', 'month'])['text'].sum().to_frame().reset_index()
time_activity['dt'] = time_activity['year'].astype(str) + '-' + time_activity['month'].astype(str)


In [89]:
fig = px.line(time_activity, x="dt", y="text", color="label", symbol="label")
fig = fig.update_layout(xaxis_title='Date', yaxis_title='# Tweets')
fig.write_html("./plots/profanity_time.html")

Use [OpenAI AI-Content Detector](https://huggingface.co/openai-community/roberta-base-openai-detector) on tweets.

In [6]:
pipe = pipeline("text-classification", model="roberta-base-openai-detector")

model.safetensors: 100%|██████████| 501M/501M [00:28<00:00, 17.5MB/s] 
Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 16.0MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 19.7MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 18.5MB/s]


[{'label': 'Real', 'score': 0.8036580085754395}]


In [14]:
sentiments = []
for tweet in tqdm(df_full_exploded['text']):
    sentiments.append(pipe(tweet))

100%|██████████| 368028/368028 [67:35:28<00:00,  1.51it/s]      


In [35]:
sentiments_scores = [sentiment[0]['score'] for sentiment in sentiments]
sentiments_labels = [sentiment[0]['label'] for sentiment in sentiments]
df_full_exploded['is_ai_generated'] = sentiments_labels
df_full_exploded['is_ai_generated'] = df_full_exploded['is_ai_generated'].map(lambda x: 0 if x == 'Real' else 1)
df_full_exploded['is_ai_generated_confidence'] = sentiments_scores
df_full_exploded['is_ai_generated_confidence'] = df_full_exploded.apply(lambda x: x['is_ai_generated_confidence'] if x['is_ai_generated'] == 1 else 1 - x['is_ai_generated_confidence'], axis=1)

df_full_exploded.head()

,user_id,label,dataset,created_at,text,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,favorite_count,favorited,retweeted,is_quote_status,is_ai_generated,is_ai_generated_confidence
0,16905397,human,botometer-feedback,2018-04-18 14:09:01+00:00,@christophkoeck In Diskussionen in D erlebe ic...,986607535674322945,9.864804e+17,927857246.0,0,0,False,False,False,0,0.000401
0,16905397,human,botometer-feedback,2018-04-18 13:59:28+00:00,"Zwei Lehrerinnen, zwölf Schüler und ein paar S...",986605134649081858,NaN,NaN,0,1,False,False,False,0,0.066825
0,16905397,human,botometer-feedback,2018-04-18 13:45:14+00:00,Antisemitismus in Deutschland =&gt; Adam Armus...,986601549194579968,NaN,NaN,2,1,False,False,False,0,0.104320
0,16905397,human,botometer-feedback,2018-04-18 08:35:02+00:00,Bayerische Politik und ihr Menschenbild – Das ...,986523487807426560,NaN,NaN,1,0,False,False,False,0,0.000454
0,16905397,human,botometer-feedback,2018-04-18 05:08:45+00:00,Es kommt doch auf den Lehrer an! #EDchatDE #Tw...,986471574818689024,9.864714e+17,16905397.0,0,0,False,False,False,0,0.003041


In [73]:
true = df_full_exploded['label'].map(lambda x: 0 if x == 'human' else 1)
pred = df_full_exploded['is_ai_generated']
cf = confusion_matrix(true, pred, labels=[0, 1])
fig = sns.heatmap(cf, annot=True, cmap=sns.cubehelix_palette(as_cmap=True), xticklabels=['Human', 'Bot'], yticklabels=['Human', 'Bot'])
plt.xlabel('AI Content Detector Label')
plt.ylabel('True Label')
fig.get_figure().savefig('./plots/ai_content_detector_results.png')
plt.close()

### Next Steps
- Ask ChatGPT to provide classification and confidence on whether tweet is bot or human generated: https://chat.openai.com/share/0d1ce518-2043-4589-92e9-2abf0e4414b5.
- Use open source ai-content detector on HF: https://huggingface.co/openai-community/roberta-base-openai-detector?text=I+like+you.+I+love+you.